In [1]:
!pip install PyPDF2 langchain faiss-cpu openai sentence-transformers

In [8]:
import os
from PyPDF2 import PdfReader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain, ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
import openai

# OpenAI API Key 설정
OPENAI_API_KEY = 'sk-'
openai.api_key = OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [9]:
# 텍스트 파일에서 텍스트 추출 함수
def extract_text_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

In [10]:
# PDF에서 텍스트 추출 함수
def extract_text_from_pdf(file_path):
    text = ""
    pdf_reader = PdfReader(file_path)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

In [11]:
# 텍스트를 청크 단위로 분할 및 벡터 스토어 생성 함수
def process_documents(text, embeddings):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    text_chunks = text_splitter.split_text(text)
    vectorstore = FAISS.from_texts(text_chunks, embeddings)
    return vectorstore

In [12]:
# 문서 기반 대화 체인 생성 함수
def get_document_conversation_chain(vectorstore):
    memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True, window_size=10)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo'),
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

In [15]:
def get_general_conversation_chain(system_prompt):
    memory = ConversationBufferWindowMemory(memory_key='history', return_messages=True, window_size=10)
    llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')
    conversation_chain = ConversationChain(
        llm=llm,
        memory=memory,
        input_key="input",
        output_key="output"
    )
    if system_prompt:
        memory.save_context({"input": system_prompt}, {"output": ""})
    return conversation_chain


In [16]:
# 파일 경로 입력
file_paths_input = input("텍스트 또는 PDF 파일 경로를 쉼표로 구분하여 입력하세요 (없으면 Enter): ")
file_paths = file_paths_input.split(',') if file_paths_input else []

In [18]:
# 파일 업로드 및 벡터 스토어 처리
all_text = ""
vectorstore = None
if file_paths:
    for file_path in file_paths:
        if file_path.endswith('.txt'):
            all_text += extract_text_from_txt(file_path)
        elif file_path.endswith('.pdf'):
            all_text += extract_text_from_pdf(file_path)

    if all_text:
        embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
        vectorstore = process_documents(all_text, embeddings)

In [19]:
# 문서 기반 대화 체인 생성
document_chain = None
if vectorstore is not None:
    document_chain = get_document_conversation_chain(vectorstore)

In [21]:
# 일반 대화 체인 생성
general_chain = get_general_conversation_chain

In [22]:
# 대화 루프
chat_history = []
terminated = False

print("질문을 입력하고 Enter 키를 누르세요. 종료하려면 '종료'를 입력하세요.")
while not terminated:
    user_input = input("질문: ")
    chat_history.append({"role": "user", "content": user_input})

    if user_input.lower() == "종료":
        terminated = True
        print("챗봇을 종료합니다. 👋")
        break

    try:
        if document_chain:
            result = document_chain({"question": user_input})
            response = result.get("answer", "응답을 생성하는 동안 오류가 발생했습니다.")
        else:
            result = general_chain({"input": user_input})
            response = result.get("output", "응답을 생성하는 동안 오류가 발생했습니다.")
        
        chat_history.append({"role": "assistant", "content": response})
        print(f"답변: {response}")
    except Exception as e:
        response = f"오류 발생: {e}"
        chat_history.append({"role": "assistant", "content": response})
        print(response)

질문을 입력하고 Enter 키를 누르세요. 종료하려면 '종료'를 입력하세요.


c:\Users\j0708\Desktop\startup0711-main\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Users\j0708\Desktop\startup0711-main\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


오류 발생: 2 validation errors for HumanMessage
content
  str type expected (type=type_error.str)
content
  value is not a valid list (type=type_error.list)
오류 발생: 2 validation errors for HumanMessage
content
  str type expected (type=type_error.str)
content
  value is not a valid list (type=type_error.list)
오류 발생: 2 validation errors for HumanMessage
content
  str type expected (type=type_error.str)
content
  value is not a valid list (type=type_error.list)
오류 발생: 2 validation errors for HumanMessage
content
  str type expected (type=type_error.str)
content
  value is not a valid list (type=type_error.list)
챗봇을 종료합니다. 👋
